In [15]:
import torchvision 
import torch
from torch.utils.data import Dataset
from torchvision import datasets
import torchvision.transforms as transforms   
import torch.optim as optim
from torchvision.transforms import RandomRotation
from torchvision.transforms import Pad
from torchvision.transforms import Resize
from torchvision.transforms import ToTensor
from torchvision.transforms import Compose
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
from PIL import Image
from numpy.linalg import norm
from torch.cuda.random import device_count
from torchvision.datasets import CIFAR10
from torch.utils.data import random_split
import torch.optim as optim
import torchvision.transforms as transforms 
import torchvision 
import math
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.4, 0.4822, 0.4465), (0.247, 0.243, 0.261))])
transform1 = transforms.Compose(
    [    
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, 4),
    transforms.ToTensor(),
    transforms.Normalize((0.4, 0.4822, 0.4465), (0.247, 0.243, 0.261)),   
    ])
trainset = torchvision.datasets.CIFAR10(root='./', train=True,download=True,transform = transform1)
validset = torchvision.datasets.CIFAR10(root='./', train=True,download=True, transform = transform) 
valid = list(range(40000, 50000,1))
validset1 = torch.utils.data.Subset(validset, valid)
validloader = torch.utils.data.DataLoader(validset1, shuffle=True, num_workers=4,batch_size = 100)
testset = torchvision.datasets.CIFAR10(root='./data', train = False, download=True, transform=transform)
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init

from torch.autograd import Variable

__all__ = ['ResNet', 'resnet20', 'resnet32', 'resnet44', 'resnet56', 'resnet110', 'resnet1202']

def _weights_init(m):
    classname = m.__class__.__name__
    #print(classname)
    if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d):
        init.kaiming_normal_(m.weight)

class LambdaLayer(nn.Module):
    def __init__(self, lambd):
        super(LambdaLayer, self).__init__()
        self.lambd = lambd

    def forward(self, x):
        return self.lambd(x)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1, option='A'):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            if option == 'A':
                """
                For CIFAR10 ResNet paper uses option A.
                """
                self.shortcut = LambdaLayer(lambda x:
                                            F.pad(x[:, :, ::2, ::2], (0, 0, 0, 0, planes//4, planes//4), "constant", 0))
            elif option == 'B':
                self.shortcut = nn.Sequential(
                     nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                     nn.BatchNorm2d(self.expansion * planes)
                )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 16

        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self._make_layer(block, 16, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 32, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 64, num_blocks[2], stride=2)
        self.linear = nn.Linear(64, num_classes)

        self.apply(_weights_init)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion

        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.avg_pool2d(out, out.size()[3])
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def resnet20():
    return ResNet(BasicBlock, [3, 3, 3])


def resnet32():
    return ResNet(BasicBlock, [5, 5, 5])


def resnet44():
    return ResNet(BasicBlock, [7, 7, 7])


def resnet56():
    return ResNet(BasicBlock, [9, 9, 9])


def resnet110():
    return ResNet(BasicBlock, [18, 18, 18])


def resnet1202():
    return ResNet(BasicBlock, [200, 200, 200])


def test(net):
    import numpy as np
    total_params = 0

    for x in filter(lambda p: p.requires_grad, net.parameters()):
        total_params += np.prod(x.data.numpy().shape)
    print("Total number of params", total_params)
    print("Total layers", len(list(filter(lambda p: p.requires_grad and len(p.data.size())>1, net.parameters()))))


if __name__ == "__main__":
    for net_name in __all__:
        if net_name.startswith('resnet'):
            print(net_name)
            test(globals()[net_name]())
            print()

resnet20
Total number of params 269722
Total layers 20

resnet32
Total number of params 464154
Total layers 32

resnet44
Total number of params 658586
Total layers 44

resnet56
Total number of params 853018
Total layers 56

resnet110
Total number of params 1727962
Total layers 110

resnet1202
Total number of params 19421274
Total layers 1202



In [4]:
def get_n_params(model):
    pp=0
    for p in list(model.fc.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp

In [10]:
    """
    model = torchvision.models.vgg19(pretrained = True)
    model.classifier[5] = nn.Sequential(
                    nn.Dropout(p=0.5, inplace=False),
                    nn.Linear(4096, 512, bias = True), 
                    nn.ReLU(), 
                    nn.Dropout(0.6, inplace=False),
                    nn.Linear(512, 256, bias = True),
                    nn.ReLU(),
                    nn.Dropout(p=0.5, inplace=False))
    for parameter in model.parameters():
        parameter.requires_grad = False
    model.classifier[6] = nn.Sequential(
                    nn.Linear(256, 64, bias = True),
                    nn.ReLU(),
                    nn.Dropout(p=0.5, inplace=False),
                    nn.Linear(64, 10))
    """
    model = resnet20()
    model.load_state_dict(torch.load("/notebooks/resnet205k1"))
    model.to(device)  
    test_model(model)
#model1 = generate_noise(model,tensor)

Test Accuracy of airplane: 83% (834/1000)
Test Accuracy of automobile: 85% (859/1000)
Test Accuracy of  bird: 56% (564/1000)
Test Accuracy of   cat: 49% (497/1000)
Test Accuracy of  deer: 72% (724/1000)
Test Accuracy of   dog: 62% (621/1000)
Test Accuracy of  frog: 72% (722/1000)
Test Accuracy of horse: 63% (639/1000)
Test Accuracy of  ship: 80% (803/1000)
Test Accuracy of truck: 79% (794/1000)

Test Accuracy (Overall): 70% (7057/10000)


0.7057

In [4]:
matrix = np.array([])
count = 1
avg = 0 
for i in range(10): 
    print("##########################################################")
    print("#########################################################")
    print("trial: " + str(i))
    indices = torch.randperm(len(trainset))[:25000]
    print(indices)
    trainset1 = torch.utils.data.Subset(trainset, indices)
    modelloader = torch.utils.data.DataLoader(trainset1, shuffle=True, num_workers=2, batch_size = 100)
    testloader = torch.utils.data.DataLoader(testset, shuffle=True, num_workers=2, batch_size = 20)
    #model pretrained by resnet
    #model = ResNet(ResidualBlock, [3, 4, 6, 3]).to(device)
    model = resnet20()
    model.load_state_dict(torch.load("/notebooks/resnet205k1"))
    model.to(device) 
    #print(get_n_params(model))
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay = 0.0001)
  ################
  #training model
  ####################
    train_losslist = []
    n_epochs = [*range(200)]
    valid_loss_min = np.Inf # track change in validation loss

    for epoch in n_epochs:
    # keep track of training and validation loss
        train_loss = 0.0
        train_loss1 = 0.0
        valid_loss = 0.0   
    ###################
    # train the model #
    ###################
        model.train()
    #model1.train()
        class_correct1 = list(0. for i in range(10))
        class_total1 = list(0. for i in range(10))
        for data, target in modelloader:
        # move tensors to GPU if CUDA is available
            data, target = data.to(device), target.to(device)
        # clear the gradients of all optimized variables
            optimizer.zero_grad()
        #optimizer1.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
        #output1 = model1(data)
        # calculate the batch loss
            loss = criterion(output, target)
        #loss1 = criterion1(output1,target)
        # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
        #loss1.backward()
        # perform a single optimization step (parameter update)
            optimizer.step()
        #exp_lr_scheduler.step()
        #optimizer1.step()
        # update training loss
            train_loss += loss.item()*data.size(0)
        #train_loss1 += loss1.item()*data.size(0)
            _, pred = torch.max(output, 1)    
        # compare predictions to true label
            correct_tensor = pred.eq(target.data.view_as(pred))
            correct = np.squeeze(correct_tensor.cpu().numpy())
        # calculate test accuracy for each object class
            for i in range(100):
                label = target.data[i]
                class_correct1[label] += correct[i].item()
                class_total1[label] += 1
    ######################    
    # validate the model #
    ######################
        model.eval()
        class_correct = list(0. for i in range(10))
        class_total = list(0. for i in range(10))
        for data, target in validloader:
            # move tensors to GPU if CUDA is available
            #if train_on_gpu:
            data, target = data.cuda(), target.cuda()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            # calculate the batch loss
            loss = criterion(output, target)
            # update average validation loss 
            valid_loss += loss.item()*data.size(0)
            # convert output probabilities to predicted class
            _, pred = torch.max(output, 1)    
            # compare predictions to true label
            correct_tensor = pred.eq(target.data.view_as(pred))
            correct = np.squeeze(correct_tensor.cpu().numpy())
            # calculate test accuracy for each object class
            for i in range(100):
                label = target.data[i]
                class_correct[label] += correct[i].item()
                class_total[label] += 1
            # calculate average losses
        valid_loss = valid_loss/len(validloader.dataset)
        train_loss = train_loss/len(modelloader.dataset)
        train_losslist.append(train_loss)
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))
        print('Training Accuracy (Overall): %2d%% (%2d/%2d)' % (
        100. * np.sum(class_correct1) / np.sum(class_total1),
        np.sum(class_correct1), np.sum(class_total1)))
        print('Validation Accuracy (Overall): %2d%% (%2d/%2d)' % (
        100. * np.sum(class_correct) / np.sum(class_total),
        np.sum(class_correct), np.sum(class_total)))
    ########################
    # testing model #
    ######################### 
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))
    test_loss = 0.
    train_on_gpu = torch.cuda.is_available()
    model.eval()
    # iterate over test data
    for data, target in testloader:
        # move tensors to GPU if CUDA is available
        data, target = data.to(device), target.to(device)
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # update test loss 
        test_loss += loss.item()*data.size(0)
        # convert output probabilities to predicted class
        _, pred = torch.max(output, 1)    
        # compare predictions to true label
        correct_tensor = pred.eq(target.data.view_as(pred))
        correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
        # calculate test accuracy for each object class
        for i in range(20):
            label = target.data[i]
            class_correct[label] += correct[i].item()
            class_total[label] += 1
    # average test loss
    test_loss = test_loss/len(modelloader.dataset)
    print('Test Loss: {:.6f}\n'.format(test_loss))

    for i in range(10):
        if class_total[i] > 0:
            print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
              classes[i], 100 * class_correct[i] / class_total[i],
              np.sum(class_correct[i]), np.sum(class_total[i])))
        else:
            print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))
    print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))
    avg += (100. * np.sum(class_correct) / np.sum(class_total))
    arr = np.array([])
    for name, param in model.named_parameters():
        temp = param.cpu().detach().numpy()
        bruh = temp.flatten()
        arr =np.concatenate((arr,bruh),axis = 0)
    arr =np.reshape(arr, (1,-1))

    print(arr.shape)
    if matrix.size == 0:
        matrix = arr
    else: 
        matrix = np.append(matrix, arr, axis = 0)
    print(count)
    torch.save(model.state_dict(), "test"+str(count))
    count = count +1
print(avg/10)
    

##########################################################
#########################################################
trial: 0
tensor([ 6167, 21986, 26286,  ..., 28249, 30278, 43063])
Epoch: 0 	Training Loss: 0.787235 	Validation Loss: 0.601171
Training Accuracy (Overall): 75% (18990/25000)
Validation Accuracy (Overall): 79% (7982/10000)
Epoch: 1 	Training Loss: 0.598482 	Validation Loss: 0.577333
Training Accuracy (Overall): 79% (19878/25000)
Validation Accuracy (Overall): 80% (8069/10000)
Epoch: 2 	Training Loss: 0.558529 	Validation Loss: 0.616633
Training Accuracy (Overall): 80% (20165/25000)
Validation Accuracy (Overall): 79% (7958/10000)
Epoch: 3 	Training Loss: 0.531316 	Validation Loss: 0.570489
Training Accuracy (Overall): 81% (20437/25000)
Validation Accuracy (Overall): 80% (8085/10000)
Epoch: 4 	Training Loss: 0.507582 	Validation Loss: 0.547770
Training Accuracy (Overall): 82% (20576/25000)
Validation Accuracy (Overall): 82% (8207/10000)
Epoch: 5 	Training Loss: 0.489118 	Va

In [6]:
np.save('10matrix5.npy', matrix)

In [13]:
def get_eig_of_differences(data):
    """
    data should be a 2-dimensional tensor whose rows are the trained parameters
    of an ML model, with one training per row. If you have a model with N
    parameters that you run M times, data should be of size (M, N). Returns a
    1-dimensional, M-element tensor whose entries are the eigenvalues (in no
    particular order) of the square matrix calculated by subtracting the mean
    of the input rows from each individual row, then right-multiplying that
    matrix by its transpose. This guarantees a M by M square matrix, which
    means that calculating the eigenvalues is surprisingly fast, even with
    reasonably large models.
    """
    mu = torch.mean(data, 0, keepdim=True).expand(data.size()[0], -1)
    deviation_data = data-mu
    square_data = torch.matmul(deviation_data, deviation_data.transpose(0, 1))
    eig = torch.linalg.eigvals(square_data)
    return eig.real

def get_gaussian_covariance(data):
    cov = torch.cov(data.t()) 
    U, S, V = torch.linalg.svd(cov)
    T = torch.nansum(torch.sqrt(S))
    for i in range(5130):
        S[i] = math.sqrt(S[i])*T
    S = torch.diag(S)
    temp = torch.matmul(U, S)
    return torch.matmul(S,V)

def get_gaussian(data):
    cov = torch.cov(data.t()) 
    U, S, V = torch.linalg.svd(cov)
    T = torch.nansum(torch.sqrt(S))
    for i in range(5130):
        S[i] = math.sqrt(S[i])*T
    return S
    
def get_l2_norm(data):
    mu = torch.mean(data, 0, keepdim=True).expand(data.size()[0], -1)
    deviation_data = data-mu
    total_norm = 0.0
    for row in deviation_data:
        norm = torch.norm(row, p=2)
        norm *= norm
        total_norm += norm
    return total_norm / data.size()[0]

def get_sample_covariance_eigs(data):
    cov = torch.cov(data.t()) # torch.cov expects columns, not rows, to be individual trials
    print(cov.size())
    #eig = torch.linalg.eigvals(cov)
    U, S, V = torch.linalg.svd(cov)
    return S
    #return eig.real


def get_eig_of_differences_fast(data, num_eigvals=-1):
    """
    ONLY USE THIS FUNCTION AFTER RUNNING AN EXCESSIVE (>10,000) NUMBER OF TRIALS
    Acts similarly to get_eig_of_differences, except that it averages the N by N
    square matrix whose eigenvalues are calculated with its transpose, then uses
    torch.lobpcg to estimate only the num_eigvals largest eigenvalues, instead
    of calculating all of them. num_eigvals defaults to data.size()[0]
    """
    mu = torch.mean(data, 0, keepdim=True).expand(data.size()[0], -1)
    deviation_data = data-mu
    square_data = torch.matmul(deviation_data, deviation_data.transpose(0, 1))
    square_data = (square_data+square_data.transpose(0, 1))/2
    eigvals_to_gen = data.size()[0] if num_eigvals == -1 else num_eigvals
    eig = torch.lobpcg(square_data, k=eigvals_to_gen)
    return eig[0]
def eig_estimation(data,q):
    #cov = torch.cov(data.t())
    U, S, V = torch.pca_lowrank(data, q, center=True, niter=2)
    return S
    

In [16]:
matrix = np.load('/notebooks/10matrix5.npy', allow_pickle = True)
data = torch.from_numpy(matrix)
print(data)
print(f"input matrix successfully loaded. It is of size {data.size()}")
eig = eig_estimation(data,10)
print(eig.size())
print(torch.nansum(torch.sqrt(eig)))
#eig = get_sample_covariance_eigs(data)
#eig = personal_eig(matrix)
#eig.to('cpu')
#print(f"Eigenvalues: {eig}")
#norm = get_l2_norm(data)
#print(norm)
#print(eig.size())
#print(torch.nansum(torch.sqrt(eig)))
#saved = torch.nansum(torch.sqrt(eig))
#torch.save('square root', matrix)
#gaussian = get_gaussian(data)
#print(gaussian)
#print(gaussian)
#np.save('gausnoise.npy', gaussian)
#print(torch.nansum(torch.sqrt(gaussian)))

tensor([[ 0.0800,  0.2814, -0.0826,  ..., -0.0333,  0.0975, -0.0510],
        [ 0.0870,  0.3085, -0.1867,  ..., -0.0077,  0.0621, -0.0981],
        [ 0.0229,  0.2229, -0.1100,  ..., -0.0192,  0.1210, -0.0866],
        ...,
        [ 0.0998,  0.1867, -0.1375,  ..., -0.0308,  0.0904, -0.0604],
        [ 0.0160,  0.2328, -0.1542,  ..., -0.0241,  0.0967, -0.0845],
        [ 0.0584,  0.1546, -0.0921,  ...,  0.0053,  0.0647, -0.0574]],
       dtype=torch.float64)
input matrix successfully loaded. It is of size torch.Size([10, 269722])
torch.Size([10])
tensor(55.7307, dtype=torch.float64)


In [17]:
def generate_gaussian_noise(cov, noise_mult=1):
    U, S, V = torch.linalg.svd(cov)
    Sprime = torch.sqrt(S)
    Sprime *= torch.nansum(Sprime)
    Sprime = torch.sqrt(Sprime)

    noise = torch.randn(cov.size()[0], device=device)*noise_mult
   # print(noise.type())
    temp = torch.diag(Sprime).type(torch.FloatTensor)
    temp = temp.to(device)
  #  print(temp.type())
    noise = torch.matmul(noise, temp)
   # print(noise.type())
    noise = noise.type(torch.DoubleTensor)
    noise = torch.matmul(U.to(device), noise.to(device))
    return noise

"""
PRIMITIVE, DON'T KNOW IF RIGHT 
"""
def gen_noise2(data,q,noise_mult=1):
    U, S, V = torch.pca_lowrank(data, q, center=True, niter=2)
    Sprime = torch.sqrt(S)
    Sprime *= torch.nansum(Sprime)
    Sprime = torch.sqrt(Sprime)
   # cov = torch.cov(data.t())
    print("here")
    noise = torch.randn(data.size()[0], device=device)*noise_mult
    print(noise.size())
    temp = torch.diag(Sprime).type(torch.FloatTensor)
    temp = temp.to(device)
    noise = torch.matmul(noise, temp)
    print(noise.size())
    noise = noise.type(torch.DoubleTensor)
    noise = torch.matmul(V.to(device), noise.to(device))
    print(noise.size())
    return noise
    
def add_noise2(model, noise):
    model1 = model
    for param in model.parameters():
        needed = param.numel()
        use, left = torch.split(noise, [needed, noise.numel()-needed])
        use.resize(param.size())
        noise1 = torch.nn.Parameter(param.cpu() + use.cpu()).to(device)
        with torch.no_grad():
            param.copy_(noise1)
        noise = left
def generate_noise(model, gaussian):
    #print(model.fc.weight)
    weights1, bias1, weights2, bias2 = torch.split(gaussian, [16384,64,640,10])
   # print(weights)
    #print(bias.size())
    #print(bias)
    #print(weights2)
    gaussianweights1 = torch.reshape(weights1, (64,256))
    gaussianweights2 = torch.reshape(weights2, (10, 64))
  #  print(gaussianweights2)
    model1 = model
    for name, param in model1.named_parameters():
        if(name == 'classifier.6.0.weight'):
            noise = torch.nn.Parameter(param.cpu() + gaussianweights1.cpu()).to(device)
            with torch.no_grad():
                param.copy_(noise)
        if(name == 'classifier.6.3.weight'):
            noise = torch.nn.Parameter(param.cpu() + gaussianweights2.cpu()).to(device)
            with torch.no_grad():
                param.copy_(noise)
        if(name == 'classifier.6.0.bias'):
            noise2 = torch.nn.Parameter(param.cpu() + bias1.cpu()).to(device)
            with torch.no_grad():
                param.copy_(noise2)
        if(name == 'classifier.6.3.bias'):
            noise2 = torch.nn.Parameter(param.cpu() + bias2.cpu()).to(device)
            with torch.no_grad():
                param.copy_(noise2)
    return model
"""
def generate_noise(model, gaussian):
    #print(model.fc.weight)
    weights, bias = torch.split(gaussian, [5120,10])
   # print(weights)
    #print(bias.size())
    #print(bias)
    
    gaussianweights = torch.reshape(weights, (10,512))
    noise = torch.nn.Parameter(model.fc.weight.cpu() + gaussianweights.cpu()).to(device)
    with torch.no_grad():
        model.fc.weight.copy_(noise)

    print(model.fc.weight)
    print(model.fc.bias)
    print(bias)
    
    noise2 = torch.nn.Parameter(model.fc.bias.cpu() + bias.cpu()).to(device)
    with torch.no_grad():
        model.fc.bias.copy_(noise2)
    
    #print(model.fc.bias)
    #print(model.fc.weight)  
    return model
"""
def test_model(model):
    test_loss = 0.0
    testloader = torch.utils.data.DataLoader(testset, shuffle=True, num_workers=2, batch_size = 20)
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))
    train_on_gpu = torch.cuda.is_available()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01)
    model.eval()
    # iterate over test data
    for data, target in testloader:
        # move tensors to GPU if CUDA is available
        data, target = data.to(device), target.to(device)
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # update test loss 
        test_loss += loss.item()*data.size(0)
        # convert output probabilities to predicted class
        _, pred = torch.max(output, 1)    
        # compare predictions to true label
        correct_tensor = pred.eq(target.data.view_as(pred))
        correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
        # calculate test accuracy for each object class
        for i in range(20):
            label = target.data[i]
            class_correct[label] += correct[i].item()
            class_total[label] += 1
    # average test loss
    test_loss = test_loss/len(testloader.dataset)
    #Q print('Test Loss: {:.6f}\n'.format(test_loss))
    
    for i in range(10):
        if class_total[i] > 0:
            
            print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
              classes[i], 100 * class_correct[i] / class_total[i],
              np.sum(class_correct[i]), np.sum(class_total[i])))
        else:
            print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))
    print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))
    return np.sum(class_correct) / np.sum(class_total)
    

In [21]:
data2 = gen_noise2(data,10)
model = resnet20()
model.load_state_dict(torch.load("/notebooks/resnet205k1"))
model.to(device)  
model = add_noise2(model, data2)

here
torch.Size([10])
torch.Size([10])
torch.Size([269722])


/usr/local/lib/python3.9/dist-packages/torch/_tensor.py:586: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


RuntimeError: requested resize to torch.Size([16, 3, 3, 3]) (torch.Size([16, 3, 3, 3]) elements in total), but the given tensor has a size of 432 (432 elements). autograd's resize can only change the shape of a given tensor, while preserving the number of elements. 

In [ ]:
#gaussian = np.load('gausnoise.npy', allow_pickle = true)
average1 = 0
average2 = 0
cov = torch.cov(data.t())
for i in range(30):
    print("trial " + str(i))
    gaussian = generate_gaussian_noise(cov)
#    print(gaussian.size())
    model = torchvision.models.vgg19(pretrained = False)
    model.classifier[5] = nn.Sequential(
                    nn.Dropout(p=0.5, inplace=False),
                    nn.Linear(4096, 512, bias = True), 
                    nn.ReLU(), 
                    nn.Dropout(0.6, inplace=False),
                    nn.Linear(512, 256, bias = True),
                    nn.ReLU(),
                    nn.Dropout(p=0.5, inplace=False))
    for parameter in model.parameters():
        parameter.requires_grad = False
    model.classifier[6] = nn.Sequential(
                    nn.Linear(256, 64, bias = True),
                    nn.ReLU(),
                    nn.Dropout(p=0.5, inplace=False),
                    nn.Linear(64, 10))
    model.load_state_dict(torch.load("/notebooks/test" + str(i+1)))
    model.to(device)
    average1 += test_model(model)
    model1 = generate_noise(model, gaussian)
    #print(test_model(model))
    average2 += test_model(model1)
print(average1/30)
print(average2/30)



trial 0
Test Accuracy of airplane: 82% (823/1000)
Test Accuracy of automobile: 89% (894/1000)
Test Accuracy of  bird: 72% (727/1000)
Test Accuracy of   cat: 73% (731/1000)
Test Accuracy of  deer: 79% (791/1000)
Test Accuracy of   dog: 64% (644/1000)
Test Accuracy of  frog: 85% (851/1000)
Test Accuracy of horse: 88% (887/1000)
Test Accuracy of  ship: 91% (919/1000)
Test Accuracy of truck: 74% (748/1000)

Test Accuracy (Overall): 80% (8015/10000)
Test Accuracy of airplane: 82% (822/1000)
Test Accuracy of automobile: 88% (885/1000)
Test Accuracy of  bird: 72% (724/1000)
Test Accuracy of   cat: 72% (724/1000)
Test Accuracy of  deer: 79% (799/1000)
Test Accuracy of   dog: 65% (658/1000)
Test Accuracy of  frog: 82% (829/1000)
Test Accuracy of horse: 88% (880/1000)
Test Accuracy of  ship: 92% (924/1000)
Test Accuracy of truck: 75% (753/1000)

Test Accuracy (Overall): 79% (7998/10000)
trial 1
Test Accuracy of airplane: 82% (823/1000)
Test Accuracy of automobile: 89% (894/1000)
Test Accuracy of